# Test03: test code for operations (t,matmul,sigmoid,relu,elu,leakyrelu,relu6)

# Preparation 

## Preparation of google drive (if required)

In [ ]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [ ]:
%cd "ctorch211122/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch211122/MiniCTorch_Prototype/notebook


## Installation of MiniCTorch

In [1]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 5.4 MB/s 


In [2]:
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

  Cloning https://github.com/kojima-r/MiniCTorch_Prototype.git to /tmp/pip-req-build-iy91lwt0
  Running command git clone -q https://github.com/kojima-r/MiniCTorch_Prototype.git /tmp/pip-req-build-iy91lwt0
  Created wheel for minictorch: filename=minictorch-0.0.1-py3-none-any.whl size=16013 sha256=2b915acf7c519139217e36de5c896fe5ebecfeaee71661675f2baea28605a4d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ry1nnse3/wheels/1c/0a/b1/3618887a2db533ac9c7b971dc10589e3976893cdf9a74ca6da
Successfully built minictorch


## ニューラルネットの定義 (活性化関数は置き換えて下さい）

In [3]:
import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch.generator as GN
import minictorch.converter as CV

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.t = torch.tensor(np.array([[5.0,6,7], [8,9,10]]))
        #self.b = torch.tensor([1.0,2.,3.],dtype=torch.float64)

    def forward(self, x):
        tt=torch.t(self.t)
        z1=torch.matmul(x,tt)
        #z2=torch.sigmoid(z1)
        z2=torch.relu(z1)
        #z2 = torch.tanh(z1)
        #fn = torch.nn.ELU()
        #fn = torch.nn.LeakyReLU()
        #fn = torch.nn.ReLU6()
        #z2 = fn(z1)
        #z2,_= torch.max( z1, 0 )
        #print(z1)
        #print(z2)
        return z2

プロジェクト名、作業フォルダ、JSONファイル名の設定

In [6]:
project = "test03"
folder = "./test"
json_path = folder + '/' + project +'.json'

In [7]:
! mkdir -p test

計算グラフをJSONファイルに出力する

In [8]:
def generate_json( json_path, input_x ):
    model = Net()
    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_x, json_path )
    return model

In [9]:
input_to_model = torch.tensor(np.array([[1.0, 2, 3]]), requires_grad=True)

model = generate_json( json_path, input_to_model )

[JSON] ./test/test03.json


JSONファイルをC++コードに変換する

In [10]:
CV.convert_cpp_code( project, folder, model, input_to_model, json_path )

[JSON] ./test/test03.json
[PARAM] ./test/test03_param.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [1, 3], 'out': [3], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [2, 3], 'constant_value': [5.0, 6.0, 7.0, 8.0, 9.0, 10.0], 'out': [2], 'sorted_id': 1}
{'name': 'Net/tt', 'op': 'aten::t', 'in': [1], 'output_id': 0, 'shape': [3, 2], 'out': [3], 'sorted_id': 2}
{'name': 'Net/z1', 'op': 'aten::matmul', 'in': [0, 2], 'output_id': 0, 'shape': [1, 2], 'out': [4], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'aten::relu', 'in': [3], 'output_id': 0, 'shape': [1, 2], 'out': [5], 'sorted_id': 4}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [4], 'output_id': 0, 'shape': [1, 2], 'out': [], 'sorted_id': 5}
[CPP]  ./test/test03.cpp
[MAKE] ./test/Makefile


pythonコードを実行する

In [11]:
# check code
model.train()
y2 = model( input_to_model )
print(y2)
y2.sum().backward()
print( "input", input_to_model.grad )

tensor([[38., 56.]], dtype=torch.float64, grad_fn=<ReluBackward0>)
input tensor([[13., 15., 17.]], dtype=torch.float64)


## 環境構築
(注意) インストール済みの場合は各自の環境に合わせて変更して下さい。

In [12]:
! git clone https://github.com/xtensor-stack/xtensor.git
! git clone https://github.com/xtensor-stack/xtensor-blas.git
! git clone https://github.com/xtensor-stack/xtl.git
! git clone https://github.com/nlohmann/json.git


Cloning into 'xtensor'...
remote: Enumerating objects: 20418, done.
remote: Counting objects: 100% (3167/3167), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 20418 (delta 2722), reused 2921 (delta 2638), pack-reused 17251
Receiving objects: 100% (20418/20418), 10.07 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (14570/14570), done.
Cloning into 'xtensor-blas'...
remote: Enumerating objects: 3857, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 3857 (delta 83), reused 96 (delta 41), pack-reused 3688
Receiving objects: 100% (3857/3857), 4.13 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (3155/3155), done.
Cloning into 'xtl'...
remote: Enumerating objects: 2399, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 2399 (delta 125), reused 137 (delta 58), pack-reused 2141
Receiving objects: 100% (2399/2399), 722.73 KiB | 12.46

In [13]:
! git clone https://github.com/kojima-r/MiniCTorch_Prototype.git

Cloning into 'MiniCTorch_Prototype'...
remote: Enumerating objects: 728, done.
remote: Counting objects: 100% (728/728), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 728 (delta 499), reused 583 (delta 367), pack-reused 0
Receiving objects: 100% (728/728), 42.78 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (499/499), done.


C++コードをコンパイル、リンクする

## C++コードをコンパイル、実行

In [14]:
C_SRC = './test/test03.cpp ./test/test03_param.cpp'
C_OPT = '-I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include '
C_OUT = './test/test03'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test03.cpp ./test/test03_param.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test03


In [15]:
!./test/test03

### forward computation ...
{{ 38.,  56.}}
### backward computation ...
input_grad{{ 13.,  15.,  17.}}
